In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from __future__ import unicode_literals

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from collections import Counter
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import nltk
# NLTK Stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, HdpModel, LdaModel


import spacy
try:
    from spacymoji import Emoji
except:
    !pip install spacymoji
    from spacymoji import Emoji

nlp = spacy.load("en_core_web_sm", disable=['parser'])
emoji = Emoji(nlp, merge_spans=False)
nlp.add_pipe(emoji, first=True)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

#pd.options.plotting.backend = "plotly"
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Utils

In [ ]:
## Time Features

def get_timefeatures(df: pd.DataFrame, col:str):
    from pandas.api.types import is_datetime64_ns_dtype as is_datetime
    from pandas.api.types import is_float_dtype as is_float
    
    if is_datetime(df[col]):
        df[col+'_date'] = df[col].dt.date
        df[col+'_doy'] = df[col].dt.day_of_year
        df[col+'_dow'] = df[col].dt.day_of_week
        df[col+'_time'] = df[col].dt.time
        df[col+'_hour'] = df[col].dt.hour
        df[col+'_week'] = df[col].dt.week
        df[col+'_day'] = df[col+'_doy'] - df[col+'_doy'].min()
        df[col+'_floorT'] = df[col].dt.floor('Min')
        df[col+'_floorH'] = df[col].dt.floor('H')
        return [col,col+'_date',col+'_doy', col+'_dow',
                col+'_time',col+'_hour',col+'_week',col+'_day', col+'_floorT', col+'_floorH']
    elif is_float(df[col]):
        df[col+'_age'] = df[col].max() - df[col]
        return [col, col+'_age']
    
## Feature Engineering

def remove_outliers(df: pd.DataFrame):
    try:
        return df[df.timestamp_doy<273]
    except:
        print('It is not possible to filter on day of year column.')
        return df
def VaderSentiment(text: str):
        d = sid.polarity_scores(text)
        return d['pos'],d['neu'],d['neg'],d['compound']
def NumbersFromText(txt: str):
    try:
        doc = nlp(txt)
        words = [token.text for token in doc if token.is_alpha]
        emojis = [token.text for token in doc if token._.is_emoji]
        return len(words), len(emojis)
    except:
        return 0,0

def get_TextFeatures(df: pd.DataFrame):
    df['full_text'] = df['title'] + ' ' + df['body'].fillna('')
    df['text_lenght'] = df['full_text'].apply(lambda x: len(x))
    p = df.full_text.transform(VaderSentiment).to_list()
    df[['VS_pos','VS_neu','VS_neg','compound']] = pd.DataFrame(p, columns=['VS_pos','VS_neu','VS_neg','compound'])
    df['sentiment'] = df['compound'].apply(lambda c: 'pos' if c>0.1 else 'neg' if c<-0.1 else 'neu')
    df[['num_of_words','num_of_emojis']] = pd.DataFrame(df.title.apply(lambda x: NumbersFromText(x)).to_list(), columns=['num_of_words','num_of_emojis'])

def extract_emoji(sent):
    doc = nlp(sent)
    res = ' '.join([token.text for token in doc if token._.is_emoji])
    return res.strip()

## Text Preprocessing

def remove_url(text):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
def remove_newline(text):
    return re.sub('\s+', ' ', text)
def remove_squote(text):
    return re.sub("\'", "", text)

def sent_to_words(texts):
    texts = [remove_url(text) for text in texts]
    texts = [remove_newline(text) for text in texts]
    texts = [remove_squote(text) for text in texts]
    for sent in texts:
        yield([token.lemma_.lower() for token in nlp(sent) if (token._.is_emoji)|(token.is_alpha)|(token.is_digit)])
#    for sent in texts:
#        yield(gensim.utils.simple_preprocess(str(sent), deacc=True))

def remove_stopwords(texts):
    return [[word for word in text if word not in stop_words] for text in texts]
#[[word for word in doc if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV','PROPN'], is_emoji=True):
    """https://spacy.io/api/annotation"""
    texts_out = []
    item = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if (token.pos_ in allowed_postags)|(is_emoji&token._.is_emoji)])
    return texts_out

# equivalent to build_corpus but optimized with spacy
def preprocess(texts: pd.Series, allowed_postags:list):
    docs = list(nlp.pipe(texts))
    data_words = [[token.lemma_.lower() for token in doc if (
        (token._.is_emoji)|(token.is_alpha)|(token.is_digit)|(token.is_stop==False)
    )&(token.pos_ in allowed_postags)]for doc in docs]
    #data_words = remove_stopwords(data_words)
    return data_words

def build_corpus(texts:pd.Series, dictionary, allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV','PROPN']):
    docs = list(nlp.pipe(texts))
    data_words = [[token.lemma_.lower() for token in doc if (
        (token._.is_emoji)|(token.is_alpha)|(token.is_digit)|(token.is_stop==False)
    )&(token.pos_ in allowed_postags)]for doc in docs]
    
    data_words = make_bigrams(data_words)
    corpus = [id2word.doc2bow(text) for text in data_words]
    return corpus

### other utils


# deprecated thanks to MultiLabelBinarizer

def in_set(stock:object, set_stock:set):
    if type(stock)==list:
        v = []
        for s in stock:
            if s in set_stock:
                v.append(1)
            else:
                v.append(0)
        return v
    elif type(stock)==str:
        if stock in set_stock:
            return 1
        else:
            return 0

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics,
                                               alpha='auto',
                                               eta='auto')#learn asymmetric priors
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Explorative Data Analysis

In [ ]:
path = '/kaggle/input/reddit-wallstreetsbets-posts/reddit_wsb.csv'
df = pd.read_csv(path, parse_dates = ['timestamp'], infer_datetime_format=True)
print(df.info())

The dataset has a total of 8 columns. From [an official page on GitHub of the API Wrapper](https://github.com/reddit-archive/reddit/wiki/JSON) we have the following:
- **title**: the title of the link. may contain newlines for some reason
- **score**:the net-score of the link. note: A submission's score is simply the number of upvotes minus the number of downvotes. If five users like the submission and three users don't it will have a score of 2. Please note that the vote numbers are not "real" numbers, they have been "fuzzed" to prevent spam bots etc. So taking the above example, if five users upvoted the submission, and three users downvote it, the upvote/downvote numbers may say 23 upvotes and 21 downvotes, or 12 upvotes, and 10 downvotes. The points score is correct, but the vote totals are "fuzzed".
- **id**: this item's identifier, e.g. "8xwlg"
- **url**: the link of this post. the permalink if this is a self-post
- **comms_num**: the number of comments that belong to this link. includes removed comments.
- **created**: the time of creation in local epoch-second format
- **body**:  the raw text. this is the unformatted text which includes the raw markup characters such as ** for bold. <, >, and & are escaped.
- **timestamp**: datetime about the related activity 

In [ ]:
import gc

gc.collect()

get_TextFeatures(df)

timefeatures = []

timefeatures.extend(get_timefeatures(df,'timestamp'))

keyfeatures=['timestamp_day','timestamp','full_text','url','score','comms_num']

In [ ]:
df.head(10)

# Questions

We need to understand what data we have since some points are still not clear.
- ~~Which is the time range?~~
- ~~Number of posts?~~
- Score and number of comments distribution and trend
- ~~Do we have posts evolution alogn time?~~

In [ ]:
df.describe(include='all')

In [ ]:
df[timefeatures].describe(include='all')

The dataset contains posts from Reddit starting from 29th January 2021. Be aware that data don't start before GME stock hype. Moreover, this dataset is regularly updated and total number of rows increases. 

The first date is an "outlier" that I've decided to remove from this analysis. 

In [ ]:
a = df[df.timestamp_doy==273][['full_text','timestamp','url']].values[0]
print('Posted at ',a[1])

print(a[0])

print('URL: ', a[2])

In [ ]:
df = remove_outliers(df)

timefeatures.extend(get_timefeatures(df,'created'))
print('Records in reddit_wsb goes from ',df.timestamp.min(),' to ',df.timestamp.max())
print('There are ',df.shape[0],'records.')

In [ ]:
fig,ax = plt.subplots(nrows=1, ncols=3, figsize=(20,5))
ax[0].set_yscale('log')
ax[1].set_yscale('log')
ax[2].set_yscale('log')
df.hist(column=['score'], bins = 25, ax = ax[0])
df.hist(column=['comms_num'], bins = 25, ax = ax[1])
df.hist(column=['text_lenght'], bins = 25, ax = ax[2])

Can we find some relationship between day/hour and reddit score, number of comments or lenght of texts? In the following we show box plot on log scale grouped by day of year or hour. 

In [ ]:
nrows = 3
ncols = 2

fig,ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (30,10))
sns.boxplot(data = df, x='timestamp_doy', y='score', ax = ax[0,0])
sns.boxplot(data = df, x='timestamp_hour', y='score', ax = ax[0,1])
sns.boxplot(data = df, x='timestamp_doy', y='comms_num', ax = ax[1,0])
sns.boxplot(data = df, x='timestamp_hour', y='comms_num', ax = ax[1,1])
sns.boxplot(data = df, x='timestamp_doy', y='text_lenght', ax = ax[2,0])
sns.boxplot(data = df, x='timestamp_hour', y='text_lenght', ax = ax[2,1])

for i in range(nrows):
    for j in range(ncols):
        ax[i,j].set_yscale('log')

fig.tight_layout()

Here is top 5 reddit posts by score.

In [ ]:
df.sort_values(by='score', ascending = False).head(5)[keyfeatures]

Here is top 5 reddit posts by number of comments.

In [ ]:
df.sort_values(by='comms_num', ascending = False).head(5)[keyfeatures]

Let's see what happend along the days. We show number of reddits, median score or number of comments by hour.

In [ ]:
df_series = df.groupby(['timestamp_doy','sentiment']).agg(
    {'id':'count','score':'median','comms_num':'median','created_age':'median','VS_pos':'median','VS_neg':'median','compound':'median'}
).reset_index()
df_series=df_series[df_series.sentiment!='neu']


nrows = 2
ncols = 3

fig,ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (20,8))

sns.lineplot(data = df_series,x='timestamp_doy',y='id',hue='sentiment', ax = ax[0,0])
sns.lineplot(data = df_series,x='timestamp_doy',y='score',hue='sentiment', ax = ax[0,1])
sns.lineplot(data = df_series,x='timestamp_doy',y='comms_num',hue='sentiment',  ax = ax[0,2])
sns.lineplot(data = df_series,x='timestamp_doy',y='VS_pos',hue='sentiment',  ax = ax[1,0])
sns.lineplot(data = df_series,x='timestamp_doy',y='VS_neg',hue='sentiment',  ax = ax[1,1])
sns.lineplot(data = df_series,x='timestamp_doy',y='compound',hue='sentiment',  ax = ax[1,2])

for i in range(ncols):
    for j in range(nrows):
        for label in ax[j,i].get_xticklabels():
            label.set_rotation(45)
fig.tight_layout()

## Stock Analysis

Is it possible to detect stock trends from WallStreetBets? In this subsection we try to understand if we can retrieve this information from our dataset.

In [ ]:
%%time

def is_stock(token):
    return (token.is_lower|token.is_upper)&(token.pos_=='PROPN')&(
    1<len(token.text)<=4)&(token._.is_emoji==False)&(token.ent_type_ in ['','ORG','PERSON'])&(token.is_stop==False)

docs = list(nlp.pipe(df.full_text))
stock_docs = [[token.text for token in doc if is_stock(token)] for doc in tqdm(docs)]
del docs

The most commont "stocks" in our dataset are the following. Be aware of two things:
- we count in how many documents a stock is cited in
- the chosen method detect stocks **but** also common words. I've still not thought about how to refine this point yet. 

In [ ]:
n_head= 20
stock_count = pd.DataFrame.from_dict(
    dict(Counter(
        [stock for stock_list in map(set, stock_docs) for stock in stock_list]
    )), orient='index'
).reset_index()
stock_count.rename(columns={'index':'stock',0:'count'},inplace=True)
stock_count = stock_count[stock_count['count']>10]

#fig,ax=plt.subplots()
stock_count.sort_values(by='count',ascending=False)[:n_head].plot.bar(x='stock',y='count',rot=0, figsize=(15,6))

In [ ]:
def filter_by_list(stdocs:list, sl:list):
    return [[stock.upper() for stock in los if stock in sl] for los in stdocs]

stock_list = stock_count[stock_count['count']>30].stock.to_list()

df['stocks'] = list(map(set,filter_by_list(stock_docs, stock_list)))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

gc.collect()

stock_trend = df['stocks'].copy()

mlb = MultiLabelBinarizer()
matrix = mlb.fit_transform(stock_trend)
stock_list = mlb.classes_
stock_trend = pd.DataFrame(matrix, columns=mlb.classes_)
stock_trend = pd.concat([stock_trend, df[['timestamp','created','id','score','comms_num','compound']]],axis = 1).dropna()

del mlb, matrix


stocks_dict = {'score':'sum','compound':'median','comms_num':'sum','flg':'sum'}

get_timefeatures(stock_trend,'timestamp')

stock_trend = stock_trend.melt(id_vars=['timestamp','id','score','compound',
                                        'comms_num','timestamp_date','timestamp_doy'],
                 value_vars=stock_list,
                 var_name='stock',
                 value_name='flg'
                )

false_stock = ['/U','HOLD','WSB','MOON','BUY','HOLD']

stock_trend = stock_trend[(stock_trend.flg>0)&(stock_trend.stock.isin(false_stock)==False)]
stock_trend = stock_trend.groupby(['timestamp_date','timestamp_doy','stock']).agg(stocks_dict).reset_index()

In [ ]:
import plotly.express as px

fig = px.line(stock_trend, x='timestamp_date',y='flg',
              color='stock', hover_name='stock'
             )
fig.update_xaxes(
        tickangle = 45,
        title_text = "Date")
fig.update_yaxes(
        title_text = "Number of Posts")
fig.show()

#stock_trend.plot.line(x='timestamp_date',y='flg', color='stock', rot=45, figsize=(15,6))

In [ ]:
import plotly.express as px

fig = px.line(stock_trend, x='timestamp_date',y='score',
              color='stock', hover_name='stock'
             )
fig.update_xaxes(
        tickangle = 45,
        title_text = "Date")
fig.update_yaxes(
        title_text = "Score Level")
fig.show()



In [ ]:
import plotly.express as px

fig = px.line(stock_trend, x='timestamp_date',y='comms_num',
              color='stock', hover_name='stock'
             )
fig.update_xaxes(
        tickangle = 45,
        title_text = "Date")
fig.update_yaxes(
        title_text = "Overall Number of Comments")
fig.show()



To Do:
- ~~improve visualization (with plotly?)~~
- correlation between stocks? - To do in another Notebook or last section
- ~~correlation between stocks, number of post, number of comments, overall score and median compound~~
- ~~define engagement~~
- animated scatterplot score vs comms_num

# Sentiment Analysis on Full Text

- ~~LDA Topic Modeling~~
- ~~Topic Interpretability~~
- ~~Sentiment Analysis based on Emoji... WIP~~

One of the most interesting things of Reddit posts is the strong presence of emojis. 
Emojis are ideograms and smilies used in message and provide their own meaning which can be that can be really different from the original one. In this section I will try to detect topics from reddits text and understand how to use emojis to detect emotions and/or topics.

In [ ]:
df['emojis'] = df.title.transform(extract_emoji)
c = Counter([emoji for sent in df.emojis.to_list() for emoji in set(sent) if emoji!=' '])
#most_common_emojis

Here I will show the the most used emojis in posts' titles.
If you have read something on WallStreetBets, these have a real clear meaning:
- 🚀 usually refer to "Going to the moon" and refers to the fact that a stock grows
- 💎, together with 🙌, means "Diamonds Hands". It measure how likely a trader is to sell if the price drops or if they make a modest profit. Diamonds means they will not drop. Otherwise they have "paper" hands.
- 🦍,= "apes together strongs" and seems to derive directly from Rise of Planet of Apes. The "apes" are the retail investors that, if they are united, they could be strong enough to outlast those short on the stock. 
- ✋ and hands-related emojis could be related to diamond and paper hands or other supportive messages (?)

Emojis could be used together or multiple times. In my opinion, in this case they should be considered as regular words.

In [ ]:
most_common_emojis = pd.DataFrame.from_dict(
    dict(c.most_common()), orient='index', columns=['count']
)
most_common_emojis = most_common_emojis[most_common_emojis['count']>1].index[:7].to_list()
most_common_emojis

In [ ]:
def emoji_mood(emojis: str):
    x = Counter([emoji for emoji in emojis.replace(' ','') if emoji in most_common_emojis]).most_common(1)
    if len(x)>=1:
        return x[0][0]
    else:
        return ''


dataset = df[['timestamp','title','emojis','sentiment','comms_num','compound']][df['emojis']!='']
dataset['emoji_mood'] = dataset.emojis.apply(emoji_mood)
dataset = dataset[dataset['emoji_mood']!='']
_ = get_timefeatures(dataset,'timestamp')
#dataset.head(5)

## LDA Topic Modeling

In [ ]:
lda_model = LdaModel.load('../input/yolo-topic-modeling/Models/lda_model')
id2word = gensim.corpora.Dictionary.load('../input/yolo-topic-modeling/Dictionary/dictionary')
bigram = gensim.models.Phrases.load('../input/yolo-topic-modeling/bigram')
bigram_mod = gensim.models.phrases.Phraser.load('../input/yolo-topic-modeling/Models/bigram_mod')

In [ ]:
%%time
corpus = build_corpus(df.full_text, id2word)

## Topic Interpretability

In [ ]:
%%time
def format_topics_sentences(texts,ldamodel=lda_model, corpus=corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: x[1], reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num, topn=7)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(
    texts=df.full_text,
    ldamodel=lda_model, 
    corpus=corpus)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
%%time
# Group top 5 sentences under each topic
sent_topics_sorteddf = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf = pd.concat([sent_topics_sorteddf,
                                      grp.sort_values(['Perc_Contribution'], 
                                                      ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf

In [ ]:
%%time
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[
    ['Dominant_Topic', 'Topic_Keywords']
].drop_duplicates()

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords.sort_values(by='Dominant_Topic').reset_index() , 
                                topic_counts.sort_index()
                                , topic_contribution.sort_index()
                               ], axis=1).drop(columns='index')

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.sort_values(by='Num_Documents', ascending=False)

### Add features from LDA model

In [ ]:
topics_docs= list(lda_model.get_document_topics(corpus, minimum_probability=0.0))
lda_topics = [sorted(topics, key=lambda x: x[1], reverse=True)[0] for topics in topics_docs]
df = pd.concat([df,pd.DataFrame(lda_topics, columns=['lda_topic','lda_topic_prob'])], axis = 1)
del topics_docs, lda_topics

In [ ]:
for num in range(lda_model.num_topics):
    print('\n')
    print('###### Topic no.',num)
    print('\n')
    for i,row in df[(df.lda_topic==num)&(df.lda_topic_prob)>0.95].sort_values(by='lda_topic_prob', ascending=False).head(3).iterrows():
        print('At: ', row.timestamp)
        print('Text: ',row.full_text)
        print('Url :', row.url)
        print('DataFrame index: ', i)
        print('\n')

In [ ]:
map_dict = {
    5:'YOLO',
    0:'Market Thoughts',
    1:'Market Analysis (?)',
    2:'Brokerage Account',
    3:'Trading Discussion (?)',
    4:'General (?)'
}

df.lda_topic = df.lda_topic.map(map_dict)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(20,20))
sns.lineplot(data=df, x='timestamp_doy',y='compound', hue='lda_topic', ax = ax[0])
sns.lineplot(data=df, x='timestamp_doy',y='score', hue='lda_topic', ax = ax[1])
sns.lineplot(data=df, x='timestamp_doy',y='comms_num', hue='lda_topic', ax = ax[2])
plt.show()

## Clustering

In [ ]:
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline

data = df.copy()
features = ['score','comms_num','created_age','compound']

pt = PowerTransformer()
data[features] = pd.DataFrame(
    pt.fit_transform(
        data[features])
)

In [ ]:
from sklearn.cluster import KMeans

sns.set_theme(style="ticks")

n_clusters = 3


data.dropna(inplace=True)

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(data[features])
data['cluster'] = kmeans.predict(data[features])
sns.pairplot(data=data[features+['cluster']], hue='cluster')
data[features] = pt.inverse_transform(data[features])
plt.show()

In [ ]:
for i in range(n_clusters):
    for text in data[data.cluster==i].full_text.head(1):
        print('Cluster : ',i)
        print('\n')
        print(text)
        print('\n')


### To Do:
- Try another tecnique
- Look optimal number of clusters

# Get Financial Historical Data 

In [ ]:
from pandas_datareader import data

tickers = ['GME']
start_date = str(df.timestamp_date.min() + pd.DateOffset(-20))
end_date = str(df.timestamp_date.max())
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

panel_data = data.DataReader(tickers, 'yahoo', start_date, end_date)
panel_data = panel_data.reindex(all_weekdays).fillna(method='ffill')

output_path = './GME_20210110.csv'
panel_data.to_csv(output_path)
panel_data['Close'].plot.line()

In [ ]:
panel_data.head(10)

To Do:
- Can we find data on minute level?

# Get News Data

In [ ]:
try:
    from GoogleNews import GoogleNews
except:
    !pip install GoogleNews
    from GoogleNews import GoogleNews

In [ ]:
googlenews = GoogleNews()
googlenews.set_lang('en')
start_date = (df.timestamp_date.min() + pd.DateOffset(0)).strftime('%d/%m/%y')
end_date = (df.timestamp_date.min()).strftime('%d/%m/%y')
googlenews.set_time_range(start_date,end_date)
googlenews.set_encode('utf-8')

In [ ]:
googlenews.get_news('GME')

In [ ]:
news = pd.DataFrame.from_records(googlenews.results())
news.to_csv('./news.csv')
news.head(10)
